# Alyce : An A.I fine tuned Screenplay writer

### Project Setting Up Section

If you are experiencing any problem while running the below code then Goto the '**Runtime**' menu and select '**Factory Reset Runtime**' to reset the notebook.

In [0]:
!git clone https://github.com/16cs009/alyce.git
import os
os.chdir("/content/alyce")
!pip install -r requirements.txt
!python -m spacy download en
import neuralcoref
import nltk
!python -m nltk.downloader all

from transformers.modeling_gpt2 import GPT2LMHeadModel
_ = GPT2LMHeadModel.from_pretrained("gpt2-medium")

### Running Alyce Server

##### Alyce Package Phase I

In [0]:
import spacy
import neuralcoref
import sys
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from Datasets import datasets

nlp = spacy.load('en')
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

def phaseI_processStory(story):
	dialogues = re.findall(r'\"(.+?)\"',story)
	dialogue_count = 0
	for dialogue in dialogues:
		qt_mark = "\"" if(story.find("\""+dialogue+"\"")!=-1) else '\''
		story = story.replace(qt_mark+dialogue+qt_mark,"<-DIALOGUE-#"+str(dialogue_count)+"#->")
		dialogue_count = dialogue_count + 1

	storylist = []
	for paragraph in story.split("\n"):
		for sentence in paragraph.split("."):
			if(len(sentence.strip())!=0):
				tmp_slist = list()
				tmp_slist.append(sentence)
				if(sentence.find("<-DIALOGUE-#")!=-1):
					tmp_slist.append("DIALOGUE")
				else:
					tmp_slist.append("NARRATION")
				tmp_slist.append("NO")
				tmp_slist.append(" ")
				tmp_slist.append(" ")
				tmp_slist.append(" ")
				storylist.append(tmp_slist)
		storylist.append(["\\n","NEWLINE"," "," "," "])

	for i in range(len(storylist)):
		if(storylist[i][1]=="DIALOGUE" and i>0):
			words = storylist[i][0].upper().split(" ")
			if("HE" in words or "SHE" in words or "HIM" in words or "HER" in words):
				j=i-1
				while(j>0):
					if(storylist[j][1]!="NEWLINE"):
						break
					j=j-1
				coreference_resolver_result = nlp(".".join([storylist[j][0],storylist[i][0]]))
				storylist[i][0] = coreference_resolver_result._.coref_resolved.split(".")[-1]
				storylist[i][2] = "YES"
				storylist[i][3] = str(coreference_resolver_result._.coref_clusters)

	k = 0
	s1 = {}
	for i in range(len(storylist)):
		if(storylist[i][1]=="DIALOGUE"):
			dialogue_holder = re.findall(r'<-DIALOGUE-#[0-9][0-9]*#->',storylist[i][0])[0]
			dialogue_containing_resolved_sentence = storylist[i][0].replace(dialogue_holder,"").strip()
			storylist[i][5] = dialogues[k]
			stop_words = set(stopwords.words("english"))
			d1 = nltk.word_tokenize(dialogue_containing_resolved_sentence.lower())
			d2 = [j for j in d1 if not j in stop_words]
			for w in nltk.pos_tag(d2):
				if(w[1]=="NN"):
					if(w[0].lower() in s1.keys()):
						s1[w[0].lower()] = s1[w[0].lower()]+1
					else:
						s1[w[0].lower()] = 1
			k = k + 1
	character_list = [k for k in s1.keys() if s1[k]>1]

	k = 0
	for i in range(len(storylist)):
		if(storylist[i][1]=="DIALOGUE"):
			storylist[i][4]="UNKNOWN"
			sentence = " ".join([ w if w not in datasets.wordList else "said" for w in storylist[i][0].split(" ")])
			for subsentence in sentence.split(","):
				if(subsentence.find("said")!=-1):
					for word in subsentence.split(" "):
						if(word.strip().lower() in character_list):
							storylist[i][4]=word.strip()
							break
					break
			k = k + 1
	return storylist

def convert_to_html(output):
  responseString = ""
  for d in output:
    if(d[1]=="NARRATION"):
      responseString = responseString + d[0]
    elif(d[1]=="NEWLINE"):
      responseString = responseString + "<br/>"
    else:
      responseString = responseString + "<br/><font color='red'><b>"+d[4]+"</b> : "+d[5]+"</font>"
      ad = ""
      for t in d[0].split(","):
        if(t.lower().find("said")==-1 and t.find("<-DIALOGUE-")==-1):
          ad = ad + t
      if(len(ad)>0):
        responseString = responseString + "<font color='green'>(("+ad+"))</font>"
      responseString = responseString+"</br>"
  return responseString

In [0]:
import os
#from alyce import Alyce
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template

app = Flask(__name__, static_url_path='', static_folder=(os.getcwd()+'/public'))
run_with_ngrok(app)

@app.route('/', methods=['GET', 'POST'])
def index():
	if(request.method == "POST"):
		story = request.form.get("story")
		if(story == None):
			return "Please Enter a Story"
		else:
			data = phaseI_processStory(story)
			response = convert_to_html(data)
			return response
	else:
		return render_template('index.html')

if __name__ == '__main__':
	app.run()

### PPLM

The below given is an example to demonstarte the PPLM in action.

In [0]:
from PPLM.run_pplm import run_pplm_example
run_pplm_example(
    cond_text="The potato",
    num_samples=3,
    bag_of_words='military',
    length=50,
    stepsize=0.03,
    sample=True,
    num_iterations=3,
    window_length=5,
    gamma=1.5,
    gm_scale=0.95,
    kl_scale=0.01,
    verbosity='regular'
)